In [2]:
#! pip install keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 3.3 MB/s eta 0:00:00m eta 0:00:010:00:010m


In [ ]:
! pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━ 337.7/585.9 MB 7.8 MB/s eta 0:00:32:34

In [3]:
# Import all packages required

import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

%matplotlib inline

# For Data Preprocessing
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# For building the LSTM Model
from keras.models import Sequential
from keras.layers import LSTM, Dense

# For evaluating the model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Import data
df = pd.read_csv('/home/consolee/Desktop/AIMS-Project/Kigali_PM2.5.csv')
df

In [ ]:
df1=df.drop(['Site','Parameter','Year','Month','Day','Hour','AQI Category','Conc. Unit','Duration','QC Name','NowCast Conc.','AQI'],axis=1)
df1

In [ ]:
data=pd.read_csv('/home/consolee/Desktop/AIMS-Project/data.csv')
data

In [ ]:
data.isnull().sum()

In [ ]:
data.plot(x='Date', y='Raw Conc.', figsize=(12,6))
plt.title('PM2.5 Time Series')
plt.ylabel('PM2.5 Concentration')
plt.show()

In [ ]:
# Convert date column to numerical values
data['Date'] = pd.to_datetime(data['Date']).astype(np.int64) // 10**9

In [ ]:
# Split the dataset into training and testing sets
train_size = int(len(data) * 0.8)
train_data = data.iloc[:train_size, :]
test_data = data.iloc[train_size:, :]

In [99]:
# Scale the dataset
scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

In [119]:
# Preprocess the dataset for LSTM
def create_dataset(X, y, time_steps=1, n_features=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:i+time_steps])
        ys.append(y[i+time_steps])
    return np.array(Xs), np.array(ys)

In [120]:
time_steps = 10
X_train, y_train = create_dataset(train_data, train_data[:, 0], time_steps)
X_test, y_test = create_dataset(test_data, test_data[:, 0], time_steps)

In [121]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(50, activation='sigmoid', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [122]:
# Train the LSTM model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

Epoch 1/50
186/186 [==============================] - 1s 2ms/step - loss: 0.0059 - val_loss: 3.3740e-06
Epoch 2/50
186/186 [==============================] - 0s 2ms/step - loss: 2.8031e-06 - val_loss: 1.4741e-06
Epoch 3/50
186/186 [==============================] - 0s 2ms/step - loss: 1.0105e-06 - val_loss: 4.5714e-07
Epoch 4/50
186/186 [==============================] - 0s 2ms/step - loss: 2.8049e-07 - val_loss: 1.0430e-07
Epoch 5/50
186/186 [==============================] - 0s 2ms/step - loss: 7.6169e-08 - val_loss: 3.5706e-08
Epoch 6/50
186/186 [==============================] - 0s 2ms/step - loss: 3.4059e-08 - val_loss: 2.9102e-08
Epoch 7/50
186/186 [==============================] - 0s 2ms/step - loss: 2.6194e-08 - val_loss: 2.5939e-08
Epoch 8/50
186/186 [==============================] - 0s 2ms/step - loss: 2.3643e-08 - val_loss: 2.3235e-08
Epoch 9/50
186/186 [==============================] - 0s 2ms/step - loss: 2.1528e-08 - val_loss: 2.0899e-08
Epoch 10/50
186/186 [===========

In [137]:
# Evaluate the LSTM model on the testing set
mse = model.evaluate(X_test, y_test, verbose=0)
print("Mean Squared Error:", mse)

Mean Squared Error: 1.97826957446523e-05


In [138]:
def create_dataset(X, y, time_steps=1, n_features=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X[i:i+time_steps, :]
        Xs.append(v)
        ys.append(y[i+time_steps])
    # If we don't have enough rows to create a dataset
    # with `time_steps` rows, return a single sequence
    if len(Xs) == 0:
        return X.reshape(1, time_steps, n_features), np.array([])
    return np.array(Xs), np.array(ys)